<a href="https://colab.research.google.com/github/skillox1/PADSW06MDCD/blob/main/Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Base de Dados Olist

In [1]:
import pandas as pd

In [5]:
import zipfile

In [8]:
zf = zipfile.ZipFile('/content/archive.zip') 
df_orders = pd.read_csv(zf.open('olist_orders_dataset.csv'))
df_order_items = pd.read_csv(zf.open('olist_order_items_dataset.csv'))
df_customers = pd.read_csv(zf.open('olist_customers_dataset.csv'))
df_sellers = pd.read_csv(zf.open('olist_sellers_dataset.csv'))
df_products = pd.read_csv(zf.open('olist_products_dataset.csv'))
df_order_payments = pd.read_csv(zf.open('olist_order_payments_dataset.csv'))
df_order_reviews = pd.read_csv(zf.open('olist_order_reviews_dataset.csv'))
df_geolocation = pd.read_csv(zf.open('olist_geolocation_dataset.csv'))



In [11]:
dfs = [df_orders,df_order_items,df_customers,df_sellers,df_products,df_order_payments,df_order_reviews,df_geolocation]

In [17]:
for x in dfs:
  print(x.shape[0])

99441
112650
99441
3095
32951
103886
99224
1000163


In [27]:
df_geolocation.geolocation_zip_code_prefix.nunique()

19015